# 此程序对应BINet文章的第一和第二个实验，可用于求解多边形区域上的Laplace方程

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy.special as scp
from torch.autograd import Variable as v
import time

In [6]:
# 设置边界条件
k = 1
def exact_sol(a,b):
    return torch.exp(k*a)*torch.sin(k*b) #对应文章例1的算例
    #return 1-abs(b)+1-abs(a)            #对应文章例2的算例

In [7]:
# 设置多边形区域和边界上的节点和边界条件
# vertex_num  多边形边数
# vertex      多边形顶点（按逆时针连接）
# M           在每条边上取的样本点数
# normal      样本点处的外法向量
# h           每条边上两点间的距离
# sample_num  总样本点数即 每条边上样本点数*多边形顶点数
# sample_x    样本点坐标
# sample_u    样本点处的函数值，即边界条件

M = 100
line = (torch.linspace(0,1-1/M,M)).reshape(-1,1)
vertex_num = 4
vertex = torch.Tensor([[-1,-1],[1,-1],[1,1],[-1,1],[-1,-1]])

rot = torch.Tensor([[0,-1],[1,0]])  #顺时针旋转90度

normal = torch.zeros(vertex_num,2)  #各边法向量
h = torch.zeros(vertex_num,1)     #各边小区间单位长度
for i in range(vertex_num):
    normal[i,:] = (vertex[i+1,:]-vertex[i,:])@rot
    normal[i,:] = normal[i,:]/(normal[i,:].norm())
    h[i] = (vertex[i+1,:]-vertex[i,:]).norm()/M

sample_num = vertex_num*M
sample_x = torch.zeros(vertex_num*M,2)
for i in range(vertex_num):
    sample_x[i*M:i*M+M,:] = vertex[i,:]+(vertex[i+1,:]-vertex[i,:])*line
sample_u = torch.zeros(sample_num,1)
for i in range(sample_num):
    if (i%M != 0) and (i%M != 1) and (i%M != M-1):
        sample_u[i] = exact_sol(sample_x[i,0],sample_x[i,1])#d0 + b0*sample_x[i,0]+c0*sample_x[i,1]+a0*sample_x[i,0]*sample_x[i,1]

In [8]:
# 构造积分矩阵
# G2     基本解在样本点处的外法向导数
# A2     各积分点的权重

time11 = time.time()
G2 = torch.zeros(sample_num,sample_num)
for i in range(sample_num):
    for j in range(sample_num):
        r = sample_x[j,:] - sample_x[i,:]
        d = r.norm()
        j0 = int(j/M)
        j1 = int((j-1)/M)%vertex_num
        G2[i,j] = -1/(2*np.pi)*(r*normal[j0,:]*h[j0]+r*normal[j1,:]*h[j1]).sum()/(2*d*d)
        

A2 = torch.zeros(sample_num,sample_num)
for i in range(sample_num):
    if (i%M != 0) and (i%M != 1) and (i%M != M-1):
        sample_u[i] = exact_sol(sample_x[i,0],sample_x[i,1])#d0 + b0*sample_x[i,0]+c0*sample_x[i,1]+a0*sample_x[i,0]*sample_x[i,1]
        for j in range(sample_num):
            if i==j:
                A2[i,j] = 1/2
            else:
                A2[i,j] = -G2[i,j]
time12 = time.time()

In [9]:
# 网络结构和损失函数

# ResNet结构
# m 为神经元个数
class Net(nn.Module):
  def __init__(self,m,out):
    super(Net, self).__init__()
    self.input = nn.Linear(2,m)
    self.block1=nn.Sequential(
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
    )
    self.block2=nn.Sequential(
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
    )
    self.block3=nn.Sequential(
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
    )
    self.block4=nn.Sequential(
      nn.Linear(m,m),nn.Sigmoid(),
      nn.Linear(m,m),nn.Sigmoid(),
    )
    self.block5=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block6=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.out = nn.Linear(m,out)
  def forward(self, x):
      x = self.input(x)
      x = self.block1(x) + x
      x = self.block2(x) + x
      x = self.block3(x) + x
      x = self.block4(x) + x
      x = self.block5(x) + x
      x = self.block6(x) + x
      x = self.out(x)
      return x

#损失函数，本质即MSE loss
class Green_loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,u_exact,u_Green):
        return torch.mean(torch.pow((u_exact-u_Green),2)) 

net1 = Net(100,1) 
Green_loss_func = Green_loss()

In [10]:
#训练过程

# optimizer   优化器
# Epoch       总训练次数
# sample_h    样本点处密度函数值h(x)
# u0          样本点处数值解

optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.001)
Epoch = 5000
loss_all = torch.zeros(int(Epoch/100)+1)
time0 = time.time()
for epoch in range(Epoch+1):
    sample_h = net1(sample_x)#这个方法甚至不需要内部点
    
    u0 = -(A2@sample_h)

    loss = Green_loss_func(sample_u,u0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%1000==0:
        print('loss, epoch, computation time:','%.4f'%loss.detach().numpy(),epoch,'%.4f'%(time.time()-time0))
        time0 = time.time()

loss, epoch, computation time: 4.1300 0 1.4791
loss, epoch, computation time: 0.0001 1000 16.9806
loss, epoch, computation time: 0.0001 2000 17.5142
loss, epoch, computation time: 0.0003 3000 12.0488
loss, epoch, computation time: 0.0005 4000 13.5667
loss, epoch, computation time: 0.0001 5000 12.7240


In [11]:
#我们将在区域内部 x_in 坐标的点上测试结果的数值精度
#测试点坐标保存在 outs.txt 中，也可以随机生成
#sample = 9801
#with open('outs.txt','r') as f:
#    data = f.readlines()
#u_in = torch.zeros(sample,1)
#x_in = torch.zeros(sample,2)
#for i in range(sample):
#    x_in[i,0] = float((data[i].split())[0])
#    x_in[i,1] = float((data[i].split())[1])
#    u_in[i] = exact_sol(x_in[i,0],x_in[i,1])

# 随机生成内部的点测试结果数值精度
# x_in 内部点坐标
# u_in 内部点对应的精确解
sample = 1000
x_in = torch.rand(sample,2)*2-1
u_in = exact_sol(x_in[:,0],x_in[:,1]).reshape(-1,1)

In [12]:
# G2_in 为内部点对应的积分点的权重
G2_in = torch.zeros(sample,sample_num)
for i in range(sample):
    for j in range(sample_num):
        j0 = int(j/M)
        j1 = int((j-1)/M)%vertex_num
        r = sample_x[j,:]-x_in[i,:]
        d = r.norm()
        G2_in[i,j] = -1/(2*np.pi)*(r*normal[j0,:]*h[j0]+r*normal[j1,:]*h[j1]).sum()/(2*d*d)

In [13]:
# 积分
# u_green 为通过BINet算出来的数值解
u_green = G2_in@sample_h

# 打印相对L2误差
print('Relaive L2 error:','%.4f'%((u_green-u_in).norm()/u_in.norm()).detach().numpy())

Relaive L2 error: 0.0983
